<a href="https://colab.research.google.com/github/ali1810/Big-DATA-week1-/blob/master/pubchem_sol_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Import the required library 
import requests
from bs4 import BeautifulSoup
import re


In [ ]:
## Get the SMILES WITH COMPOUND NAME
data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
html = BeautifulSoup(data.content, "xml")
html.find_all("PC-Urn_label")
smiles_tag = html.find(name="PC-Urn_label", string="SMILES")
smiles_parents = smiles_tag.find_parent("PC-InfoData")
smiles = smiles_parents.find('PC-InfoData_value_sval').string
smiles




'CC(=O)OC1=CC=CC=C1C(=O)O'

In [ ]:
### code to get the compund solubility based on the compound id which is also called CID of the compound 
x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
num = 1
sol = []
for i in range(4):
  #print(x%num)
  data=requests.get(x % num)
  
  html = BeautifulSoup(data.content, "xml")
  solubility = html.find(name='TOCHeading', string='Solubility')
  if solubility ==None:
    solub=None
    sol.append(solub)
  else:
    solub=solubility.find_next_sibling('Information').find(name='String').string
    sol.append(solub)
  
  num += 1

print(sol)


[None, None, None, 'greater than or equal to 100 mg/mL at 67.6° F (NTP, 1992)']


In [ ]:
data =requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/1983/JSON?heading=Solubility")
#data = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
html = BeautifulSoup(data.content, "xml")
html.find_all("PC-Urn_label")



[]

In [1]:
##install pubchem python library
!pip install pubchempy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=3cb0ba9e405f291982c4310c2ed8aadb9f97725d55409fbbf8478a6ce75d152f
  Stored in directory: /root/.cache/pip/wheels/7c/3d/8c/8192697412e9899dc55bbbb08bbc1197bef333caaa2a71c448
Successfully built pubchempy


In [2]:
import pubchempy as pcp

import pandas as pd

In [ ]:
prop=pcp.get_properties([ 'MolecularWeight' 
],'C1=CC=C(C=C1)C=O', 'smiles')


In [ ]:
print(prop[0])

{'CID': 240, 'MolecularWeight': '106.12'}


In [ ]:
x = list(map(lambda x: x["CID"], prop))

In [ ]:
print(x)

[240]


In [ ]:
print(x[0])

240


In [3]:
import pubchempy as pcp
import requests
from bs4 import BeautifulSoup
### Function to get solubility data from Pubchem 
def smiles_to_sol(SMILES):
   prop=pcp.get_properties([ 'MolecularWeight'], SMILES, 'smiles')
   x = list(map(lambda x: x["CID"], prop))
   y=x[0]
   #print(y)
   x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
   data=requests.get(x % y)
   html = BeautifulSoup(data.content, "xml")
   solubility = html.find(name='TOCHeading', string='Solubility')
   #print(dtype(solubility))
   if solubility == None:
      return None  
   #elif          # or pass
   else:
     solub=solubility.find_next_sibling('Information').find(name='String').string
     return solub

In [ ]:
prop=pcp.get_properties([ 'MolecularWeight'], 'CC(=O)OC1=CC=CC=C1C(=O)O','smiles')
x = list(map(lambda x: x["CID"], prop))
y=x[0]
   #print(y)
x = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/%s/xml"
data=requests.get(x % y)
html = BeautifulSoup(data.content, "xml")
solubility = html.find(name='TOCHeading', string='Solubility')
print(type(solubility))

<class 'bs4.element.Tag'>


In [ ]:

## Taking smiles from user 
smiles = input("Enter your smiles : ")

## passing smiles to a function to get the solubility from Pubchem websites 
smiles_to_sol(smiles)


Enter your smiles : CC(=O)OC1=CC=CC=C1C(=O)O


'less than 1 mg/mL at 73° F (NTP, 1992)'

In [4]:
import pandas as pd
### Getting the list of the smiles 


df=pd.read_excel("/content/drive/MyDrive/pubchem_1_10000.xlsx")

#df

In [5]:
print(len(df))

10000


In [6]:

P_sol=[]
for i in range(len(df)-5000):
  try:
    sol=smiles_to_sol(df.SMILES[i])
    P_sol.append(sol)
  except AttributeError as e:
    sol=='No string'
    P_sol.append(sol)

   # P_sol.append(sol)  

In [7]:
print(len(P_sol))

5000


In [8]:
### Making dataframe withb solubility  dataset 
df1 = pd.DataFrame(P_sol)
df1.columns =['Solubility']
print(df1)


                                             Solubility
0                                                  None
1                                                  None
2                                                  None
3     greater than or equal to 100 mg/mL at 67.6° F ...
4                                                  None
...                                                 ...
4995                                               None
4996                                               None
4997                                               None
4998                                               None
4999                                               None

[5000 rows x 1 columns]


In [9]:
### merging the data in one dataframe with smiles and solubility 

df_new = pd.concat([df[0:5000], df1], axis=1)
df_new

,SrN,SMILES,Solubility
0,1,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,None
1,2,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,None
2,3,C1=CC(C(C(=C1)C(=O)O)O)O,None
3,4,CC(CN)O,greater than or equal to 100 mg/mL at 67.6° F ...
4,5,C(C(=O)COP(=O)(O)O)N,None
...,...,...,...
4995,5284,CC1=CC=C(OC1=O)C(=CC=CC(=C2C(=O)CC3C2(CCC4C3(C...,None
4996,5285,CC1=CC=C(OC1=O)C(=CC=CC(=C2C(=O)CC3C2(CCC4C3(C...,None
4997,5286,CC(=C1C(=O)CC2C1(CCC3C2(CCC(=O)C3(C)C)C)C)C=CC...,None
4998,5287,CC(=CC=CC(=C1C(=O)CC2C1(CCC3C2(CCC(=O)C3(C)C)C...,None


In [10]:
### Sorting the data with solubility and dicarding the null
df_n = df_new.loc[df_new['Solubility'].notnull(), ['SMILES','Solubility']]
print(len(df_n))

953


In [11]:
### Making dataframe 
df_n=pd.DataFrame(df_n)

In [12]:
print(len(df_n))

953


In [13]:
df_n

,SMILES,Solubility
3,CC(CN)O,greater than or equal to 100 mg/mL at 67.6° F ...
5,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,13.8 [ug/mL]
9,C(CCl)Cl,"5 to 10 mg/mL at 66° F (NTP, 1992)"
11,C1=CC(=C(C=C1Cl)Cl)Cl,"less than 1 mg/mL at 70Â° F (NTP, 1992)"
29,C(C=O)Cl,greater than or equal to 100 mg/mL at 66° F (N...
...,...,...
4965,CC(C)NCC(C1=CC=C(C=C1)NS(=O)(=O)C)O,7.82e-01 g/L
4976,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCCC(=O)C4=CC=C...,50.1 [ug/mL]
4977,CC1CC=CC=CC(C(CC(C(C(C(CC(=O)O1)O)OC)OC2C(C(C(...,Slightly soluble in water
4991,CC1=CN(C(=O)NC1=O)C2CCC(O2)CO,>33.9 [ug/mL]


In [14]:
sol_new=df_n.Solubility

In [15]:
import re
sol1=[]
### Using regular expression module cleaning the data and taking out the data with with unit "mg/mL" in the pubchem ... 
for s in sol_new:
   sol=re.findall(r'(?<![\-\/])(\d+(?:\.?\d+)) *mg\b', s)
   sol1.append(sol) 
print(len(sol1))

953


In [16]:
df3=df_n.SMILES

In [17]:
df3.reset_index(drop=True, inplace=True)  

In [18]:
print(type(df3))

<class 'pandas.core.series.Series'>


In [19]:
df2 = pd.DataFrame(sol1)
df2.columns =['Solubility_new']
print(len(df2))

953


In [20]:
df_new = pd.concat([df3,df2], axis=1,ignore_index=True)
df_new

,0,1
0,CC(CN)O,100
1,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,None
2,C(CCl)Cl,10
3,C1=CC(=C(C=C1Cl)Cl)Cl,None
4,C(C=O)Cl,100
...,...,...
948,CC(C)NCC(C1=CC=C(C=C1)NS(=O)(=O)C)O,None
949,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCCC(=O)C4=CC=C...,None
950,CC1CC=CC=CC(C(CC(C(C(C(CC(=O)O1)O)OC)OC2C(C(C(...,None
951,CC1=CN(C(=O)NC1=O)C2CCC(O2)CO,None


In [21]:
df_new.columns =['SMILES', 'SOLUBILITY mg/mL']


In [22]:
df_new

,SMILES,SOLUBILITY mg/mL
0,CC(CN)O,100
1,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,None
2,C(CCl)Cl,10
3,C1=CC(=C(C=C1Cl)Cl)Cl,None
4,C(C=O)Cl,100
...,...,...
948,CC(C)NCC(C1=CC=C(C=C1)NS(=O)(=O)C)O,None
949,C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCCC(=O)C4=CC=C...,None
950,CC1CC=CC=CC(C(CC(C(C(C(CC(=O)O1)O)OC)OC2C(C(C(...,None
951,CC1=CN(C(=O)NC1=O)C2CCC(O2)CO,None


In [23]:
### Final data would be having the smiles along with solubility 

df_sol = df_new.loc[df_new['SOLUBILITY mg/mL'].notnull(), ['SMILES','SOLUBILITY mg/mL']]
df_sol

,SMILES,SOLUBILITY mg/mL
0,CC(CN)O,100
2,C(CCl)Cl,10
4,C(C=O)Cl,100
5,C(CCl)O,100
6,CCC(C)C(=O)C(=O)O,915.5
...,...,...
937,COC1=C(C=CC(=C1)C2C(OC3=C(O2)C=C(C=C3)C4C(C(=O...,77.36
939,[Na+].[Cl-],100
940,[F-].[Na+],50
943,C(C(=O)O)I,100
